In [13]:
import scanpy as sc
import numpy as np
import functools
import jax
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import scanpy as sc
import gseapy as gp
import matplotlib.pyplot as plt
import gseapy as gp
import anndata as ad
import decoupler
from pathlib import Path

In [14]:
split = 3

In [15]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [16]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [17]:
adata_ood.obs.head()

,sample,Size_Factor,n.umi,RT_well,Drug1,Drug2,Well,n_genes,n_genes_by_counts,total_counts,...,split,control,cell_type,cell_line,smiles_drug_1,smiles_drug_2,ood_1,ood_2,ood_3,ood_4
Cell,,,,,,,,,,,,,,,,,,,,,
A01_A02_RT_BC_22_Lig_BC_17,sciPlex_theis,0.899999,2416,RT_22,control,Dacinostat,B10,1572,1569,2412.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_26,sciPlex_theis,0.470861,1264,RT_22,control,Dacinostat,B10,902,901,1263.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_27,sciPlex_theis,0.508485,1365,RT_22,control,Dacinostat,B10,934,934,1365.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_64,sciPlex_theis,1.300827,3492,RT_22,control,Dacinostat,B10,2104,2102,3490.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_84,sciPlex_theis,0.790852,2123,RT_22,control,Dacinostat,B10,1434,1432,2121.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood


In [18]:
adata_ood.obs["condition"].cat.categories

Index(['Panobinostat+Crizotinib', 'Panobinostat+Curcumin',
       'Panobinostat+SRT1720', 'Panobinostat+Sorafenib',
       'SRT2104+Alvespimycin', 'control', 'control+Alvespimycin',
       'control+Dacinostat'],
      dtype='object')

In [19]:
adata_train.obs["condition"].cat.categories

Index(['Alvespimycin+Pirarubicin', 'Cediranib+PCI-34051',
       'Dacinostat+Danusertib', 'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051',
       'Givinostat+Carmofur', 'Givinostat+Cediranib', 'Givinostat+Crizotinib',
       'Givinostat+Curcumin', 'Givinostat+Dasatinib', 'Givinostat+SRT1720',
       'Givinostat+SRT2104', 'Givinostat+Sorafenib', 'Givinostat+Tanespimycin',
       'Panobinostat+Alvespimycin', 'Panobinostat+Dasatinib',
       'Panobinostat+PCI-34051', 'Panobinostat+SRT2104',
       'Panobinostat+SRT3025', 'SRT3025+Cediranib', 'control',
       'control+Dasatinib', 'control+Givinostat', 'control+Panobinostat',
       'control+SRT2104'],
      dtype='object')

In [20]:
control_mean = adata_ood[adata_ood.obs["condition"]=="control"].X.mean(axis=0)
rng = np.random.default_rng(0)
drug_effect = {}
for cond in adata_train.obs["condition"].cat.categories:
    if cond=="control":
        continue
    drug_effect[cond] = adata_train[adata_train.obs["condition"]==cond].X.mean(axis=0) - control_mean

drug_effect["control"] = adata_ood[adata_ood.obs["condition"]=="control"].X.toarray()
    

In [21]:
drug_effect_ood = {}

drug_effect_ood['Panobinostat+Crizotinib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Crizotinib'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['Panobinostat+Curcumin'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Curcumin'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['Panobinostat+SRT1720'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+SRT1720'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['Panobinostat+Sorafenib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Sorafenib'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['SRT2104+Alvespimycin'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+SRT2104'] 
    - drug_effect['control+Givinostat']
    + drug_effect['Panobinostat+Alvespimycin']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['control+Alvespimycin'] = (
    drug_effect['control'] 
    + drug_effect['Panobinostat+Alvespimycin']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['control+Dacinostat'] = (
    drug_effect['control'] 
    + drug_effect['Dacinostat+Dasatinib']
    - drug_effect['control+Dasatinib']
)


In [22]:
import anndata as ad
import pandas as pd
all_data = []
conditions = []

for condition, array in drug_effect_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred_ood = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [23]:
adata_pred_ood.X = np.array(adata_pred_ood.X)

In [24]:
adata_pred_ood.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/additive/combosciplex/adata_ood_with_predictions_{split}.h5ad")